In [1]:
import pandas as pd
from pathlib import Path

# File paths
file_path = Path(r"D:\climate change\Tutorial_Climate Data_2025-26.xlsx")
monthly_output = Path(r"D:\climate change\monthly_averages.xlsx")
yearly_output = Path(r"D:\climate change\yearly_averages.xlsx")

# Load sheet names
excel = pd.ExcelFile(file_path)
sheets = excel.sheet_names

# Process each sheet & save results
with pd.ExcelWriter(monthly_output, engine='openpyxl', datetime_format='YYYY-MM-DD') as monthly_writer, \
     pd.ExcelWriter(yearly_output, engine='openpyxl', datetime_format='YYYY-MM-DD') as yearly_writer:
    
    for sheet in sheets:
        # Read sheet without header
        df = pd.read_excel(file_path, sheet_name=sheet, header=None)

        # Extract coordinates, dates, data
        lons  = df.iloc[1, 1:]
        lats  = df.iloc[2, 1:]
        dates = pd.to_datetime(df.iloc[3:, 0])
        data  = df.iloc[3:, 1:]

        # Generate station names
        station_cols = [f"Station-{i}" for i in range(1, data.shape[1] + 1)]

        # Build the DataFrame indexed by date
        data_df = pd.DataFrame(data.values, index=dates, columns=station_cols)

        # Monthly and yearly averages
        monthly_avg = data_df.resample("M").mean()     # Month-end freq
        yearly_avg  = data_df.resample("Y").mean()     # Year-end freq

        # Build output format with Lon & Lat
        def build_output(avg_df):
            out_df = pd.DataFrame(index=["Lon", "Lat"] + list(avg_df.index),
                                  columns=station_cols)
            out_df.loc["Lon"] = lons.values
            out_df.loc["Lat"] = lats.values
            out_df.loc[avg_df.index] = avg_df.values
            return out_df

        # Write results
        build_output(monthly_avg).to_excel(monthly_writer, sheet_name=f"{sheet}_monthly")
        build_output(yearly_avg).to_excel(yearly_writer, sheet_name=f"{sheet}_yearly")

print(f"✅ Monthly averages saved to: {monthly_output}")
print(f"✅ Yearly averages saved to: {yearly_output}")


C:\Users\ankit\AppData\Local\Temp\ipykernel_18748\1998810473.py:34: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_avg = data_df.resample("M").mean()     # Month-end freq
C:\Users\ankit\AppData\Local\Temp\ipykernel_18748\1998810473.py:35: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_avg  = data_df.resample("Y").mean()     # Year-end freq
C:\Users\ankit\AppData\Local\Temp\ipykernel_18748\1998810473.py:34: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_avg = data_df.resample("M").mean()     # Month-end freq
C:\Users\ankit\AppData\Local\Temp\ipykernel_18748\1998810473.py:35: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_avg  = data_df.resample("Y").mean()     # Year-end freq
C:\Users\ankit\AppData\Local\Temp\ipykernel_18748\1998810473.py:34: Fu

✅ Monthly averages saved to: D:\climate change\monthly_averages.xlsx
✅ Yearly averages saved to: D:\climate change\yearly_averages.xlsx
